<a href="https://colab.research.google.com/github/shoulhafiz/python_data_science/blob/master/Naive_Bayes_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
  !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/political-landscape/political-landscape.json


--2019-11-06 03:36:03--  https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/political-landscape/political-landscape.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2020656 (1.9M) [text/plain]
Saving to: ‘political-landscape.json’

political-landscape 100%[===================>]   1.93M  --.-KB/s    in 0.05s   

2019-11-06 03:36:03 (35.3 MB/s) - ‘political-landscape.json’ saved [2020656/2020656]



https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

In [2]:
import json

with open('political-landscape.json') as fopen:
  politics = json.load(fopen)

politics.keys()

dict_keys(['kerajaan', 'pembangkang'])

In [0]:
X = politics['kerajaan'] + politics['pembangkang']
Y = [0] * len(politics['kerajaan']) + [1] * len(politics['pembangkang']) #assign 0 to kerajaan, 1 to pembangkang


In [0]:
import re

def cleaning(string):
  string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [
                word
                for word in string.split()
                if word.find('#') < 0 and word.find('@') < 0
            ]
        ),
    )
  string = re.sub('[^A-Za-z ]+', ' ', string)
  string = re.sub(r'[ ]+', ' ', string).strip()
  return string.lower()


X[0], cleaning(X[0])

for i in range(len(X)):
  X[i] = cleaning(X[i])

In [8]:
# bow
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
from sklearn.feature_extraction.text import CountVectorizer

bow_word = CountVectorizer(ngram_range=(1, 2)).fit(X)
bow_char = CountVectorizer(ngram_range=(3, 5), analyzer='char').fit(X)

X_bow_word = bow_word.transform(X)
X_bow_char = bow_char.transform(X)

X_bow_word.shape, X_bow_char.shape


((22577, 160054), (22577, 191850))

In [9]:
# tfidf
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_word = TfidfVectorizer(ngram_range=(1, 2)).fit(X)
tfidf_char = TfidfVectorizer(ngram_range=(3, 5), analyzer='char').fit(X)

X_tfidf_word = tfidf_word.transform(X)
X_tfidf_char = tfidf_char.transform(X)

X_tfidf_word.shape, X_tfidf_char.shape

((22577, 160054), (22577, 191850))

In [0]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(X_bow_word, Y, test_size = 0.2)


In [0]:
# https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB

from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB


In [0]:
nb_bow_word = MultinomialNB().fit(train_X, train_Y)
predict_Y = nb_bow_word.predict(test_X)

In [20]:
print(
    metrics.classification_report(
        test_Y, predict_Y, target_names = ['kerajaan', 'pembangkang'],
        digits = 5
    )
)

              precision    recall  f1-score   support

    kerajaan    0.95638   0.93795   0.94707      3062
 pembangkang    0.87442   0.90990   0.89181      1454

    accuracy                        0.92892      4516
   macro avg    0.91540   0.92393   0.91944      4516
weighted avg    0.92999   0.92892   0.92928      4516



In [21]:
string = ['saya suka najib', 'saya suka dap']

new_bow_word = bow_word.transform(string)
nb_bow_word.predict_proba(new_bow_word)

array([[0.01459739, 0.98540261],
       [0.097845  , 0.902155  ]])

In [0]:
train_X, test_X, train_Y, test_Y = train_test_split(X_bow_char, Y, 
                                                    test_size = 0.2)

nb_bow_char = MultinomialNB().fit(train_X, train_Y)
predict_Y = nb_bow_char.predict(test_X)


In [23]:
print(
    metrics.classification_report(
        test_Y, predict_Y, target_names = ['kerajaan', 'pembangkang'],
        digits = 5
    )
)

              precision    recall  f1-score   support

    kerajaan    0.95684   0.94733   0.95206      3019
 pembangkang    0.89587   0.91383   0.90476      1497

    accuracy                        0.93623      4516
   macro avg    0.92636   0.93058   0.92841      4516
weighted avg    0.93663   0.93623   0.93638      4516



In [24]:
string = ['saya suka najib', 'saya suka dap']

new_bow_char = bow_char.transform(string)
nb_bow_char.predict_proba(new_bow_char)

array([[1.71932963e-05, 9.99982807e-01],
       [2.21548442e-02, 9.77845156e-01]])